In [ ]:
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text

In [ ]:
!pip install transformers datasets

In [ ]:
!pip install -q tensorflow-text

In [ ]:
from transformers import TFBertModel, BertTokenizer
import tensorflow as tf

In [ ]:
import pandas as pd

In [ ]:
df=pd.read_csv('combined_data.csv')

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    df["text"],
    df["label"],
    test_size=0.2,
    random_state=42,
    stratify=df["label"]
)


In [ ]:
# Tokenize (convert to list!)
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

train_encodings = tokenizer(X_train.tolist(), truncation=True, padding=True, max_length=128, return_tensors="tf")
test_encodings = tokenizer(X_test.tolist(), truncation=True, padding=True, max_length=128, return_tensors="tf")

# Convert labels
y_train = tf.convert_to_tensor(y_train.values)
y_test = tf.convert_to_tensor(y_test.values)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [ ]:
train_dataset = tf.data.Dataset.from_tensor_slices((
    {
        "input_ids": train_encodings["input_ids"],
        "attention_mask": train_encodings["attention_mask"]
    },
    y_train
)).shuffle(1000).batch(16)

test_dataset = tf.data.Dataset.from_tensor_slices((
    {
        "input_ids": test_encodings["input_ids"],
        "attention_mask": test_encodings["attention_mask"]
    },
    y_test
)).batch(16)


In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Dropout, Layer
from tensorflow.keras.models import Model
from transformers import TFBertModel, BertTokenizer

# Load BERT model
bert_model = TFBertModel.from_pretrained("bert-base-uncased")

# Custom wrapper layer
class BertLayer(Layer):
    def __init__(self, **kwargs):
        super(BertLayer, self).__init__(**kwargs)
        self.bert = bert_model

    def call(self, inputs):
        input_ids, attention_mask = inputs
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        return outputs.pooler_output

# Inputs
input_ids = Input(shape=(128,), dtype=tf.int32, name="input_ids")
attention_mask = Input(shape=(128,), dtype=tf.int32, name="attention_mask")

# BERT Layer
bert_output = BertLayer()([input_ids, attention_mask])

# Additional layers
x = Dropout(0.3)(bert_output)
x = Dense(256, activation="relu")(x)
x = Dropout(0.3)(x)
x = Dense(64, activation="relu")(x)
output = Dense(1, activation="sigmoid")(x)

# Model
model = Model(inputs=[input_ids, attention_mask], outputs=output)
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=2e-5),
              loss="binary_crossentropy",
              metrics=["accuracy"])
model.summary()

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions w

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_ids           │ (None, 128)       │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ attention_mask      │ (None, 128)       │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ bert_layer          │ (None, 768)       │          0 │ input_ids[0][0],  │
│ (BertLayer)         │                   │            │ attention_mask[0… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout (Dropout)   │ (None, 768)       │          0 │ bert_layer[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, 256)       │    196,864 │ dropout[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_1 (Dropout) │ (None, 256)       │          0 │ dense[0][0]       │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_1 (Dense)     │ (None, 64)        │     16,448 │ dropout_1[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_2 (Dense)     │ (None, 1)         │         65 │ dense_1[0][0]     │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 213,377 (833.50 KB)

 Trainable params: 213,377 (833.50 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
model.fit(
    train_dataset,
    validation_data=test_dataset,
    epochs=10
)

Epoch 1/10
4173/4173 ━━━━━━━━━━━━━━━━━━━━ 652s 152ms/step - accuracy: 0.6195 - loss: 0.6443 - val_accuracy: 0.7976 - val_loss: 0.4529
Epoch 2/10
4173/4173 ━━━━━━━━━━━━━━━━━━━━ 625s 150ms/step - accuracy: 0.7649 - loss: 0.4914 - val_accuracy: 0.8363 - val_loss: 0.3731
Epoch 3/10
4173/4173 ━━━━━━━━━━━━━━━━━━━━ 623s 149ms/step - accuracy: 0.7972 - loss: 0.4395 - val_accuracy: 0.8679 - val_loss: 0.3180
Epoch 4/10
4173/4173 ━━━━━━━━━━━━━━━━━━━━ 623s 149ms/step - accuracy: 0.8177 - loss: 0.4055 - val_accuracy: 0.8732 - val_loss: 0.3027
Epoch 5/10
4173/4173 ━━━━━━━━━━━━━━━━━━━━ 624s 149ms/step - accuracy: 0.8200 - loss: 0.3962 - val_accuracy: 0.8905 - val_loss: 0.2831
Epoch 6/10
4173/4173 ━━━━━━━━━━━━━━━━━━━━ 641s 154ms/step - accuracy: 0.8274 - loss: 0.3873 - val_accuracy: 0.8924 - val_loss: 0.2769
Epoch 7/10
4173/4173 ━━━━━━━━━━━━━━━━━━━━ 665s 149ms/step - accuracy: 0.8302 - loss: 0.3798 - val_accuracy: 0.9009 - val_loss: 0.2645
Epoch 8/10
4173/4173 ━━━━━━━━━━━━━━━━━━━━ 625s 150ms/step - ac

In [ ]:
# Evaluate on the test dataset
loss, accuracy = model.evaluate(test_dataset)

print(f"Test Loss: {loss:.4f}")
print(f"Test Accuracy: {accuracy:.4f}")

1044/1044 ━━━━━━━━━━━━━━━━━━━━ 124s 119ms/step - accuracy: 0.9000 - loss: 0.2684
Test Loss: 0.2653
Test Accuracy: 0.9026


In [ ]:
def check_email_spam():
    user_input = input("Enter the email text to check if it's SPAM or NOT SPAM:\n")

    encoding = tokenizer(
        user_input,
        truncation=True,
        padding='max_length',
        max_length=128,
        return_tensors="tf"
    )

    prediction = model.predict({
        "input_ids": encoding["input_ids"],
        "attention_mask": encoding["attention_mask"]
    }, verbose=0)

    prob = prediction[0][0]
    label = "SPAM" if prob > 0.5 else "NOT SPAM"

    print(f"\nPrediction: {label}")
    print(f"Confidence Score: {prob:.4f}")

check_email_spam()

Enter the email text to check if it's SPAM or NOT SPAM:
hii lets meet today

Prediction: SPAM
Confidence Score: 0.9665


In [ ]:
model.export("model/saved_model")
tokenizer.save_pretrained("model/tokenizer")

Saved artifact at 'model/saved_model'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): List[TensorSpec(shape=(None, 128), dtype=tf.int32, name='input_ids'), TensorSpec(shape=(None, 128), dtype=tf.int32, name='attention_mask')]
Output Type:
  TensorSpec(shape=(None, 1), dtype=tf.float32, name=None)
Captures:
  136955714396112: TensorSpec(shape=(), dtype=tf.resource, name=None)
  136955714396496: TensorSpec(shape=(), dtype=tf.resource, name=None)
  136955714397264: TensorSpec(shape=(), dtype=tf.resource, name=None)
  136955714400336: TensorSpec(shape=(), dtype=tf.resource, name=None)
  136955714399760: TensorSpec(shape=(), dtype=tf.resource, name=None)
  136955714398032: TensorSpec(shape=(), dtype=tf.resource, name=None)
  136955714399184: TensorSpec(shape=(), dtype=tf.resource, name=None)
  136955714400912: TensorSpec(shape=(), dtype=tf.resource, name=None)
  136955714397648: TensorSpec(shape=(), dtype=tf.resource, name=None)
  136955714399952: Ten

('model/tokenizer/tokenizer_config.json',
 'model/tokenizer/special_tokens_map.json',
 'model/tokenizer/vocab.txt',
 'model/tokenizer/added_tokens.json')

In [ ]:
# Zip the SavedModel directory
!zip -r saved_model.zip model/saved_model

# Zip the tokenizer directory
!zip -r tokenizer.zip model/tokenizer

  adding: model/saved_model/ (stored 0%)
  adding: model/saved_model/assets/ (stored 0%)
  adding: model/saved_model/fingerprint.pb (stored 0%)
  adding: model/saved_model/saved_model.pb (deflated 93%)
  adding: model/saved_model/variables/ (stored 0%)
  adding: model/saved_model/variables/variables.index (deflated 76%)
  adding: model/saved_model/variables/variables.data-00000-of-00001 (deflated 7%)
  adding: model/tokenizer/ (stored 0%)
  adding: model/tokenizer/special_tokens_map.json (deflated 42%)
  adding: model/tokenizer/tokenizer_config.json (deflated 75%)
  adding: model/tokenizer/vocab.txt (deflated 53%)


In [ ]:
from google.colab import files

# Download the model zip
files.download("saved_model.zip")

# Download the tokenizer zip
files.download("tokenizer.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
model.save("model/email_spam_model.keras")  # saves as a Keras native format